In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
orders = pd.read_csv('https://raw.githubusercontent.com/joaolcorreia/RFM-analysis/master/sample-orders.csv',encoding = "ISO-8859-1")
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franzsisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


In [3]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [4]:

# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [14]:

rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                           'order_id': lambda x: len(x),      # Frequency
                                           'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

In [15]:

rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


In [16]:
(NOW - dt.datetime(2013,11,11)).days==415

True

In [17]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [18]:
quantiles

,recency,frequency,monetary_value
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


In [19]:
quantiles = quantiles.to_dict()

In [20]:
quantiles

{'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0},
 'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0}}

In [33]:

rfmSegmentation = rfmTable

In [34]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [35]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [36]:

rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))

In [37]:
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))

In [38]:
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [48]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)
        
rfmSegmentation['RFMSCORE']=rfmSegmentation.R_Quartile.astype(int)\
                            + rfmSegmentation.F_Quartile.astype(int)\
                            + rfmSegmentation.M_Quartile.astype(int)

In [49]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,RFMSCORE
customer,,,,,,,,
Aaron Bergman,415,3,887,1,1,1,111,3
Aaron Hawkins,12,7,1744,4,3,2,432,9
Aaron Smayling,88,7,3050,2,3,3,233,8
Adam Bellavance,54,8,7756,3,3,4,334,10
Adam Hart,34,10,3249,3,4,3,343,10


In [44]:
rfmSegmentation[rfmSegmentation['RFMClass']=='444'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Sanjit Engle,9,11,12210,4,4,4,444
John Lee,21,11,9801,4,4,4,444
Pete Kriz,9,12,8647,4,4,4,444
Harry Marie,2,10,8237,4,4,4,444
Lena Creighton,16,12,7661,4,4,4,444
Patrick O'Brill,4,11,7474,4,4,4,444
Brenda Bowman,28,9,6766,4,4,4,444
Dan Reichenbach,3,9,6528,4,4,4,444
James Galang,1,11,6367,4,4,4,444


In [58]:
rfmsc1=rfmSegmentation.groupby('RFMSCORE').agg({'recency': lambda x:x.sum(),
                                                'frequency': lambda x:x.sum(),
                                                'monetary_value': lambda x:x.sum()
                                               })
rfmsc1.head()

,recency,frequency,monetary_value
RFMSCORE,,,
3,30887,216,36785
4,21476,279,82476
5,18596,429,137976
6,12026,497,213223
7,10166,488,212230
